# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096952


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:02<00:41,  1.44s/it]

Rendering models:  10%|▉         | 3/31 [00:04<00:41,  1.47s/it]

Rendering models:  13%|█▎        | 4/31 [00:04<00:29,  1.08s/it]

Rendering models:  19%|█▉        | 6/31 [00:07<00:29,  1.17s/it]

...

Rendering models:  74%|███████▍  | 23/31 [00:10<00:01,  6.16it/s]

Rendering models:  77%|███████▋  | 24/31 [00:10<00:01,  6.24it/s]

Rendering models:  97%|█████████▋| 30/31 [00:10<00:00,  7.70it/s]

Rendering models: 100%|██████████| 31/31 [00:11<00:00,  6.64it/s]

not-logged-in-3b0b929f4475fd276561    0.000632
Jnursssmith5263                       0.001212
not-logged-in-673e227f920b24558cc1    0.001208
not-logged-in-f737655ff84da201cfb7    0.005922
DevionJ                               0.000368
jnarayanbvg                           0.002687
Jonas_Cross                           0.000347
awright5                              0.003205
Nauter                                0.000508
Hunting_X                             0.001779
TSM_Ginga                             0.001343
not-logged-in-a8038c9112f1e417d868    0.005346
tingard                               0.000550
Jessica_Korkmaz                       0.007191
kawaboy                               0.001034
not-logged-in-9a45911eb71184fe7c7f    0.000909
alexismfernandez                      0.000795
Planetme                              0.003341
pangeli5                              0.000668
acapirala                             0.000316
TRISTANM18                            0.000386
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())